In [1]:
import pandas as pd
import numpy as np
from glob import glob, iglob
import pymongo
import json
import os

In [47]:
#read in the final merged dataframe for building the json
data_loc=os.path.join('../data','final_df.csv')
final_df = pd.read_csv(data_loc)
final_df

,Unnamed: 0,Country,out14/15,out15/16,out16/17,out17/18,out18/19,Abv,latitude,longitude,...,pvt_fgn_bach,pvt_fgn_mast,pvt_fgn_phd,Yearly,in14/15,in15/16,in16/17,in17/18,in18/19,in19/20
0,0,Burundi,2,0,11,0,0,BI,-3.373056,29.918886,...,NaN,NaN,NaN,NaN,165,148,180,134,141,164.0
1,1,Djibouti,0,0,0,0,0,DJ,11.825138,42.590275,...,NaN,NaN,NaN,NaN,7,9,10,9,9,6.0
2,2,Eritrea,0,1,1,0,5,ER,15.179384,39.782334,...,NaN,NaN,NaN,NaN,108,109,138,151,117,81.0
3,3,Ethiopia,261,302,73,207,249,ET,9.145000,40.489673,...,NaN,NaN,NaN,NaN,1472,1517,1847,2118,2061,2356.0
4,4,Kenya,634,681,778,927,926,KE,-0.023559,37.906193,...,NaN,NaN,NaN,NaN,3072,3019,3189,3322,3451,3710.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,197,Tonga,15,11,21,21,18,TO,-21.178986,-175.198242,...,NaN,NaN,NaN,NaN,95,130,133,169,186,161.0
198,198,Tuvalu,0,0,0,0,0,TV,-7.109535,177.649330,...,NaN,NaN,NaN,NaN,7,4,3,4,4,4.0
199,199,Vanuatu,0,2,0,14,3,VU,-15.376706,166.959158,...,NaN,NaN,NaN,NaN,8,9,11,14,15,13.0
200,200,Wallis and Futuna,-,-,-,-,0,WF,-13.768752,-177.156097,...,NaN,NaN,NaN,NaN,18,4,2,0,2,0.0


In [30]:
#test that data is being pulled
# final_df[final_df['Country']== 'Tonga']['latitude'].values[0]

-21.178986

In [52]:
def myfunction(df):
    countries=df["Country"].unique()
    #empty list to hold josn results
    my_json={}
    #build json dict for pushing to MongoDB
    for country in countries:
        country_dict={
            'cord':{
                'latitude':df[df['Country']== country]['latitude'].values[0],
                'longitude':df[df['Country']== country]['longitude'].values[0]
            },
            'abv':df[df['Country']==country]['Abv'].values[0],
            'inbound':{
                'in14/15':df[df['Country']== country]['in14/15'].values[0],
                'in15/16':df[df['Country']== country]['in15/16'].values[0],
                'in16/17':df[df['Country']== country]['in16/17'].values[0],
                'in17/18':df[df['Country']== country]['in17/18'].values[0],
                'in18/19':df[df['Country']== country]['in18/19'].values[0],
                'in19/20':df[df['Country']== country]['in19/20'].values[0]
            },
            'outbound':{
                'out14/15':df[df['Country']== country]['out14/15'].values[0],
                'out15/16':df[df['Country']== country]['out15/16'].values[0],
                'out16/17':df[df['Country']== country]['out16/17'].values[0],
                'out17/18':df[df['Country']== country]['out17/18'].values[0],
                'out18/19':df[df['Country']== country]['out18/19'].values[0]
            },
            'public_avg_tuit':{
                'pub_bachelor':df[df['Country']== country]['pub_fgn_bach'].values[0],
                'pub_masters':df[df['Country']== country]['pub_fgn_mast'].values[0],
                'pub_phd':df[df['Country']== country]['pub_fgn_phd'].values[0],
            },
             'private_avg_tuit':{
                'pvt_bachelor':df[df['Country']== country]['pvt_fgn_bach'].values[0],
                'pvt_masters':df[df['Country']== country]['pvt_fgn_mast'].values[0],
                'pvt_phd':df[df['Country']== country]['pvt_fgn_phd'].values[0],
            },
            'yearly_c_o_l': df[df['Country']== country]['Yearly'].values[0],
        }
        my_json[f'{country}']=country_dict

    return my_json

In [53]:
results=myfunction(final_df)
results

{'Burundi': {'cord': {'latitude': -3.3730559999999996,
   'longitude': 29.918885999999997},
  'abv': 'BI',
  'inbound': {'in14/15': '165',
   'in15/16': '148',
   'in16/17': '180',
   'in17/18': '134',
   'in18/19': '141',
   'in19/20': 164.0},
  'outbound': {'out14/15': '2',
   'out15/16': '0',
   'out16/17': '11',
   'out17/18': '0',
   'out18/19': '0'},
  'public_avg_tuit': {'pub_bachelor': nan, 'pub_masters': nan, 'pub_phd': nan},
  'private_avg_tuit': {'pvt_bachelor': nan,
   'pvt_masters': nan,
   'pvt_phd': nan},
  'yearly_c_o_l': nan},
 'Djibouti': {'cord': {'latitude': 11.825138, 'longitude': 42.590275},
  'abv': 'DJ',
  'inbound': {'in14/15': '7',
   'in15/16': '9',
   'in16/17': '10',
   'in17/18': '9',
   'in18/19': '9',
   'in19/20': 6.0},
  'outbound': {'out14/15': '0',
   'out15/16': '0',
   'out16/17': '0',
   'out17/18': '0',
   'out18/19': '0'},
  'public_avg_tuit': {'pub_bachelor': nan, 'pub_masters': nan, 'pub_phd': nan},
  'private_avg_tuit': {'pvt_bachelor': nan,
